In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install syncode flask flask-cors pyngrok transformers accelerate torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf
from transformers import pipeline
import torch

conf.get_default().auth_token = "2wUpowdASyKkBenEMYs30JR9K7S_yKK7M2Ey62TPA5jpbL4h"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.8/205.8 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 134.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 🔧 1. Initialiser Flask
app = Flask(__name__)
CORS(app)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
@app.route("/generate", methods=["POST"])
def generate_code():
    data = request.get_json()
    prompt = data.get("prompt", "")

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ➕ Supprimer le prompt du début si nécessaire
    response_code = generated_text[len(prompt):].strip()

    return jsonify({"code": response_code})


In [ ]:
public_url = ngrok.connect(5000)
print("🔗 URL publique à utiliser dans Django :", public_url)
app.run(host="0.0.0.0", port=5000)

🔗 URL publique à utiliser dans Django : NgrokTunnel: "https://377922471237.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [14/Jul/2025 23:26:26] "POST /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [14/Jul/2025 23:27:05] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jul/2025 23:27:09] "POST /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [14/Jul/2025 23:28:05] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jul/2025 23:28:0